In [1]:
"""
Γεμίζω τον πίνακα με τις μετρήσεις
του labevent 8ώρου.

Ο πίνακας μου απαρτίζεται από 388 στήλες και 68.191 γραμμές,
οπότε μου δημιουργούν 26.458.108 κελιά.

Οι τιμές μου ανέρχονται στα 1.276.856 οπότε με μια αφαίρεση
προκείπτουν 25.181.252 NaN κελιά, πράγμα που επιβεβαιώνει
και η εντολή df_lab_tbl.isna().sum().sum()

"""

'\nΓεμίζω τον πίνακα με τις μετρήσεις\nτου labevent 8ώρου.\n\nΟ πίνακας μου απαρτίζεται από 388 στήλες και 68.191 γραμμές,\nοπότε μου δημιουργούν 26.458.108 κελιά.\n\nΟι τιμές μου ανέρχονται στα 1.276.856 οπότε με μια αφαίρεση\nπροκείπτουν 25.181.252 NaN κελιά, πράγμα που επιβεβαιώνει\nκαι η εντολή df_lab_tbl.isna().sum().sum()\n\n'

In [2]:
#Εισαγωγή βιβλιοθήκης Panda.
import pandas as pd

In [3]:
df_lab_tbl = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o14_empty_lab_tbl.csv', index_col = False)
df_data_labevents = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\hosp\d_labitems.csv.gz')
df_grouped_cerebral_infarction = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o13_cerebral_infarction_labevents_grouped_8hours_date_time.csv')

In [4]:
# Φέρνω τις τελευταίες τέσσερις στήλες πρώτες
cols = list(df_lab_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_lab_tbl = df_lab_tbl[cols]

cols = list(df_lab_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_lab_tbl = df_lab_tbl[cols]

cols = list(df_lab_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_lab_tbl = df_lab_tbl[cols]

cols = list(df_lab_tbl.columns)
cols = [cols[-1]] + cols[:-1]
df_lab_tbl = df_lab_tbl[cols]

In [5]:
# Ελέγχω αν υπάρχει NaN στην στήλη με τις μέσες τιμές, το αποτέλεσμα ισούται με μηδέν.
df_grouped_cerebral_infarction['Mean_Lab'].isna().sum()

0

In [6]:
df_grouped_cerebral_infarction = pd.merge(df_grouped_cerebral_infarction, df_data_labevents, on=['itemid'])

In [7]:
# Δημιουργώ εκ νέου την στήλη concat χωρίς κενά ανάμεσα στα στοιχεία.
# Θα χρησιμοποιήσω αυτή τη στήλη σαν index και τα κενά μπορεί να δημιουργήσουν πρόβλημα.
df_lab_tbl["concat"] = (df_lab_tbl["subject_id"].astype(str)
                          + df_lab_tbl["hadm_id"].astype(str)
                          + df_lab_tbl["date"].astype(str)
                          + df_lab_tbl["time"].astype(str))

# Δημιουργώ και την στήλη department έτσι ώστε να συμπληρωθεί το τμήμα
# από το οποίο έγινε η καταγραφή. Δεν έχει λειτουργική σημασία στο
# σημείο που είμαστε αλλά θα χρειαστεί ΄στον ενοποιημένο πίνακα.
df_lab_tbl.insert(2, 'Department', '')

In [8]:
# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat για να αποφύγω πιθανά προβλήματα
df_lab_tbl['concat'] = df_lab_tbl['concat'].str.replace('-','')
df_lab_tbl['concat'] = df_lab_tbl['concat'].str.replace(':','')

In [9]:
# Ορίζω την στήλη concat ως index των γραμμών
df_lab_tbl = df_lab_tbl.set_index('concat')

In [10]:
# Δημιουργώ την στήλη concat και στον πίνακα που περιλαμβάνει τις μετρήσεις για να είναι το κοινό σημείο ανάμεσα στους δυο πίνακες
df_grouped_cerebral_infarction["concat"] = (df_grouped_cerebral_infarction["subject_id"].astype(str)
                                            + df_grouped_cerebral_infarction["hadm_id"].astype(str)
                                            + df_grouped_cerebral_infarction["date"].astype(str)
                                            + df_grouped_cerebral_infarction["time"].astype(str))

In [11]:
# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat για να αποφύγω πιθανά προβλήματα
df_grouped_cerebral_infarction['concat'] = df_grouped_cerebral_infarction['concat'].str.replace('-','')
df_grouped_cerebral_infarction['concat'] = df_grouped_cerebral_infarction['concat'].str.replace(':','')

In [12]:
#Δημιουργώ και την στήλη Department
df_grouped_cerebral_infarction.insert(0, 'Department', 'hosp')

In [13]:
# Μεταφέρω την τελευταία στήλη πρώτη
cols = list(df_grouped_cerebral_infarction.columns)
cols = [cols[-1]] + cols[:-1]
df_grouped_cerebral_infarction = df_grouped_cerebral_infarction[cols]

In [14]:
for row in df_grouped_cerebral_infarction.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Mean_Lab
    df_lab_tbl.at['{}'.format(con), 'Department'] = row.Department
    df_lab_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [15]:
# Έλεγχος NaN περιοχών σε όλο το dataframe
df_lab_tbl.isna().sum().sum()

25181252

In [16]:
df_lab_tbl

,subject_id,hadm_id,Department,date,time,50801,50802,50803,50804,50805,...,52278,52279,52281,52283,52285,52286,52312,52369,52391,52769
concat,,,,,,,,,,,,,,,,,,,,,
100047332741187621741204160000,10004733,27411876,hosp,2174-12-04,16:00:00,NaN,0.0,NaN,26.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741204080000,10004733,27411876,hosp,2174-12-04,08:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205160000,10004733,27411876,hosp,2174-12-05,16:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741206000000,10004733,27411876,hosp,2174-12-06,00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741207000000,10004733,27411876,hosp,2174-12-07,00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199999872386574521451109000000,19999987,23865745,hosp,2145-11-09,00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451107000000,19999987,23865745,hosp,2145-11-07,00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451103080000,19999987,23865745,hosp,2145-11-03,08:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_lab_tbl.to_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o15_fill_lab_itemid_header_tbl.csv', index=False)